In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import pandas as pd
import numpy as np
from nltk import word_tokenize, sent_tokenize
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
from nltk.corpus import stopwords
from string import punctuation
from functools import reduce
import gensim
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [87]:
df = pd.read_csv('../Chubb/data/question_answers.csv')[:1000]

In [115]:
%%reload
# df['Transcript page heading'].unique()

import pysentiment as ps
ps.HIV4()

UsageError: Cell magic `%%reload` not found.


#### import stopwords

In [88]:
nlp = spacy.load('en')
df_stopwords = pd.read_csv('../Chubb/data/final_stopword list.csv')
df_stopwords[df_stopwords.flag == 0].Stop_words
stopwords = nlp.Defaults.stop_words | set(pd.read_csv('../Chubb/data/Stopwords_List_Updated.csv').stopword_list) | set(df_stopwords[df_stopwords.flag == 0].Stop_words)

#### columns for topic modeling along with number of topics

In [108]:
# [(col1, n_topics), ...]
columns_for_topic_modeling = [('Questions', 5), ('Answers', 5)]
n_grams = (1,3)
ignore_words_less_than_nORnPERCENT_documents = 1
ignore_words_more_than_nORnPERCENT_documents = 1.0


In [109]:
def convert_doc_term_matrix_to_tuples(doc_term_matrix_tf):
    """
    convert document term matrix in `tuples(word_id, count/value)`
    """
    return [[(element[1], doc_term_matrix_tf[tuple(element)]) \
             for element in np.dstack(doc_term_matrix_tf.nonzero())[0] if element[0] == idx] \
                    for idx in range(doc_term_matrix_tf.shape[0])]

cdtmtt = convert_doc_term_matrix_to_tuples
Lda = gensim.models.ldamodel.LdaModel

# countvectorizer and lda models
cv = {}
lda = {}

for col, n_topics in columns_for_topic_modeling:
    cv[col] = CountVectorizer(stop_words=stopwords, ngram_range=n_grams,
                              max_df=ignore_words_more_than_nORnPERCENT_documents, 
                              min_df=ignore_words_less_than_nORnPERCENT_documents)
    doc_term_matrix_tf = cv[col].fit_transform(df[col].fillna(''))
    dtm = cdtmtt(doc_term_matrix_tf)
    id2word = {v: k for k, v in cv[col].vocabulary_.items()}
    print('topic-modeling')
    lda[col] = Lda(dtm, num_topics=n_topics, passes=50, id2word=id2word)
    topics = dict(lda[col].print_topics())
    df[col+'_topic'] = list(lda[col][cdtmtt(cv[col].transform(df[col].fillna('')))])
    df[col+'_TOPICS'] = str(topics)
#     break



topic-modeling


In [110]:
df.to_csv('data/topic_test.csv')


In [116]:
lda[col] = Lda(dtm, num_topics=n_topics, passes=50, id2word=id2word, alpha=0.1)
topics = dict(lda[col].print_topics())
df[col+'_topic'] = list(lda[col][cdtmtt(cv[col].transform(df[col].fillna('')))])
df[col+'_TOPICS'] = str(topics)

In [117]:
topics

{0: '0.004*"growth" + 0.003*"prime" + 0.003*"margin" + 0.002*"investments" + 0.002*"amazon" + 0.002*"impact" + 0.002*"price" + 0.001*"luck" + 0.001*"fulfillment" + 0.001*"competition"',
 1: '0.003*"revenue" + 0.003*"prime" + 0.003*"growth" + 0.003*"aws" + 0.002*"impact" + 0.002*"international" + 0.002*"stores" + 0.002*"tom" + 0.002*"customers" + 0.002*"margins"',
 2: '0.006*"growth" + 0.005*"prime" + 0.003*"margin" + 0.003*"price" + 0.003*"margins" + 0.003*"international" + 0.002*"aws" + 0.002*"impact" + 0.002*"sales" + 0.002*"unit"',
 3: '0.004*"price" + 0.004*"stores" + 0.003*"investments" + 0.002*"video" + 0.002*"prime" + 0.002*"price investments" + 0.002*"sales" + 0.002*"margin" + 0.002*"growth" + 0.002*"fulfillment"',
 4: '0.006*"growth" + 0.003*"amazon" + 0.003*"international" + 0.003*"store" + 0.003*"aws" + 0.003*"price" + 0.002*"prime" + 0.002*"sales" + 0.002*"margins" + 0.002*"investment"'}